Check out Kaggle at https://www.kaggle.com/c/conways-reverse-game-of-life-2020. The objective is to reverse Conway's Game of Life Board (25x25 cells) to learn its position $\delta$ steps ago ($1<=\delta<=5$). We experiment with Convolvers, CNN Encoder-Decoders, and Generative Adverserial Networks, while evaluating performance on custom decaying losses (MSE/Brier, Entropy, Hinge) as well as ROC.

In [69]:
import pandas as pd
import numpy as np
import scipy as sp
import collections
import gc
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Lambda, Flatten, Conv2DTranspose
from tensorflow.keras.layers import Dropout, GaussianNoise, Input, UpSampling2D, Concatenate, BatchNormalization
from tensorflow.keras.models import Model, Sequential, load_model
import sklearn.model_selection
from copy import deepcopy
import sklearn.metrics

<h2>Data Preprocessing</h2>

In [19]:
start = time.time()
data = pd.read_csv("./train.csv")
X1 = data[data.columns[np.vectorize(lambda s: "stop" in s)(data.columns.values)]]
X1 = X1.values.reshape(data.shape[0],25,25,1)
delta = data["delta"].values
X2 = np.zeros((data.shape[0],5))
for i in range(data.shape[0]):
    X2[i,delta[i]-1] = 1
Y = data[data.columns[np.vectorize(lambda s: "start" in s)(data.columns.values)]]
Y = Y.values.reshape(data.shape[0],25,25,1)
print("Preprocessing Exited in "+str(time.time()-start)+" seconds")
data.shape, X1.shape, X2.shape, Y.shape

Preprocessing Exited in 6.177941083908081 seconds


((50000, 1252), (50000, 25, 25, 1), (50000, 5), (50000, 25, 25, 1))

<h2> SECTION 1: Building the Discriminator

In [68]:
def generate_real_samples(inp, out, mask, size):
    indices = np.random.choice(inp.shape[0], size=size, replace=False)
    return inp[indices], out[indices], mask[indices], mask[indices]
def generate_fake_samples(inp, out, mask, size):
    indices = np.random.choice(inp.shape[0], size=size, replace=False)
    exp = np.zeros((indices.shape[0], 5))
    oindices = np.vectorize(lambda s: s+1 if s!=(inp.shape[0]-1) else np.random.choice(inp.shape[0]-1))(indices)
    return inp[indices], out[oindices], mask[indices], exp

In [72]:
def build_discriminator(lr=2e-3):
    tf.keras.backend.clear_session()
    curimg, befimg = Input((25,25,1)), Input((25,25,1))
    imginp = Concatenate()([curimg, befimg])
    mask = Input((5,))
    #Increasing Filters to Capture Relationship
    conv1 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(imginp)
    max1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv1)
    conv2 = Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max1)
    max2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv2)
    #BatchNormalization for Train-Time Stability
    max2 = BatchNormalization()(max2)
    #Reducing Filters to 5-Channel Layer
    conv3 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max2)
    max3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv3)
    fconv = Conv2D(filters=5, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max3)
    #Discriminate on Relevant Delta
    final = Dense(5, activation="sigmoid")(Flatten()(fconv))
    final = tf.math.multiply(final,mask)
    model = Model(inputs=[curimg, befimg, mask], outputs=final)
    model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(lr=lr))
    return model

In [57]:
np.random.seed(1)
R1, R2, R3, R = generate_real_samples(X1[:40000],Y[:40000],X2[:40000], 20000)
F1, F2, F3, F = generate_fake_samples(X1[:40000],Y[:40000],X2[:40000], 20000)
indices=np.random.choice(40000,40000,replace=False)
I1, I2 = np.concatenate([R1,F1])[indices], np.concatenate([R2,F2])[indices]
I3, O = np.concatenate([R3,F3])[indices], np.concatenate([R,F])[indices]
I1.shape, I2.shape, I3.shape, O.shape

((40000, 25, 25, 1), (40000, 25, 25, 1), (40000, 5), (40000, 5))

In [58]:
np.random.seed(1)
VR1, VR2, VR3, VR = generate_real_samples(X1[40000:],Y[40000:],X2[40000:], 5000)
VF1, VF2, VF3, VF = generate_fake_samples(X1[40000:],Y[40000:],X2[40000:], 5000)
indices=np.random.choice(10000,10000,replace=False)
VI1, VI2 = np.concatenate([VR1,VF1])[indices], np.concatenate([VR2,VF2])[indices]
VI3, VO = np.concatenate([VR3,VF3])[indices], np.concatenate([VR,VF])[indices]
VI1.shape, VI2.shape, VI3.shape, VO.shape

((10000, 25, 25, 1), (10000, 25, 25, 1), (10000, 5), (10000, 5))

In [59]:
model = build_discriminator()
model.fit([I1,I2,I3],O, epochs=10, batch_size=256, validation_data=([VI1,VI2,VI3],VO))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 29s 726us/sample - loss: 0.0135 - val_loss: 0.0013
Epoch 2/10
40000/40000 [==============================] - 30s 758us/sample - loss: 0.0015 - val_loss: 0.0010
Epoch 3/10
40000/40000 [==============================] - 32s 792us/sample - loss: 0.0011 - val_loss: 8.9325e-04
Epoch 4/10
40000/40000 [==============================] - 31s 774us/sample - loss: 8.6685e-04 - val_loss: 0.0010
Epoch 5/10
40000/40000 [==============================] - 32s 803us/sample - loss: 7.1406e-04 - val_loss: 8.1544e-04
Epoch 6/10
40000/40000 [==============================] - 31s 786us/sample - loss: 6.3483e-04 - val_loss: 8.4389e-04
Epoch 7/10
 7424/40000 [====>.........................] - ETA: 24s - loss: 5.0311e-04

KeyboardInterrupt: 